In [20]:
import numpy as np
import torch  
import torch.nn as nn

def compress_matrix(x):

    if len(x.shape) != 2:
        A, B, C, D = x.shape
        x = x.reshape(A * B,  C * D)
        ind = np.argwhere((np.sum(np.abs(x), axis=1)) == 0.)
        # remove non-necessary filters and rows
        x = x[:, (x != 0).any(axis=0)]
        x = x[(x != 0).any(axis=1), :]
        x = x.reshape(-1,B,C,D)
    else:
        # remove unnecessary rows, columns
        x = x[(x != 0).any(axis=1), :]
        x = x[:, (x != 0).any(axis=0)]
        ind = numpy.array([0.]) # for now
                          
    return x, ind











model = sparsify(LeNet())

new_channels = []

for m in model.children():
    if isinstance(m, nn.Conv2d):
        num_channels = compress_layer(m)
        new_channels.append(num_channels)

compressed = TinyNet(channels=new_channels)

for layer, compressed_layer in zip(model.children(), compressed.children()):
    compressed_layer.weight = layer.weight
    if(layer.bias):
        compressed_layer.bias   = layer.bias







example = nn.Conv2d(3,6,5)

print(example)

x = example.weight.data.numpy()
x[1,:] = 0.

x, ind = compress_matrix(x)
out_, in_, height_, width_ = x.shape

new_layer = nn.Conv2d(in_, out_, height_)
new_layer.weight.data = torch.Tensor(x)

print(new_layer)

## remove the bias


# now need to remove the inputs corresponding to the channel we just pruned in the following layer
# could be conv, bn, pool or fc
# layer 2 of LeNet
next_layer = nn.Conv2d(6,16,5)



NameError: name 'compress_layer' is not defined

In [93]:
l1 = nn.Conv2d(2,3,2)
print(l1)
l1 = l1.weight.data.numpy()
l1[1,:] = 0. 
l1, ind = compress_matrix(l1)
out_, in_, height_, width_ = l1.shape

new_layer = nn.Conv2d(in_, out_, height_)
new_layer.weight.data = torch.Tensor(l1)

print(new_layer)



# follow up 
x = nn.Conv2d(3,3,2)
x = x.weight.data.numpy()

print(x)

print("\n\n\n")

print(ind)
x = np.delete(x, ind, 0)


print(x)

Conv2d (2, 3, kernel_size=(2, 2), stride=(1, 1))
Conv2d (2, 2, kernel_size=(2, 2), stride=(1, 1))
[[[[ 0.00356248  0.0972721 ]
   [-0.02880004  0.12900484]]

  [[-0.2342647   0.1986678 ]
   [ 0.2438885  -0.15374212]]

  [[-0.28073126 -0.2048888 ]
   [ 0.03231326 -0.0389329 ]]]


 [[[-0.08958508  0.11959311]
   [ 0.2767036  -0.18248624]]

  [[-0.10308418  0.1094003 ]
   [-0.2624169  -0.17878887]]

  [[-0.05216798 -0.27589944]
   [-0.13807479  0.1349493 ]]]


 [[[-0.25280896  0.12326646]
   [ 0.25075805  0.0231232 ]]

  [[-0.24188693 -0.06609032]
   [-0.06584451  0.26322126]]

  [[ 0.1502811   0.13606629]
   [-0.00836688 -0.08792694]]]]




[[2]
 [3]]
[[[[ 0.00356248  0.0972721 ]
   [-0.02880004  0.12900484]]

  [[-0.2342647   0.1986678 ]
   [ 0.2438885  -0.15374212]]

  [[-0.28073126 -0.2048888 ]
   [ 0.03231326 -0.0389329 ]]]


 [[[-0.08958508  0.11959311]
   [ 0.2767036  -0.18248624]]

  [[-0.10308418  0.1094003 ]
   [-0.2624169  -0.17878887]]

  [[-0.05216798 -0.27589944]
   [-0.1380

/home/jack/miniconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


In [15]:
x = nn.Conv2d(3,2,2).weight.data.numpy()


print(x, "\n\n--------")


print(x[1,:,:,:], "\n\n---------")
print(x[:,1,:,:], "\n\n---------")

[[[[-0.11309339 -0.13252267]
   [ 0.10189015  0.23333943]]

  [[ 0.18838969 -0.20607772]
   [-0.04675874 -0.2579743 ]]

  [[-0.22457686 -0.12578286]
   [ 0.10464424 -0.02261609]]]


 [[[ 0.21070072  0.12532869]
   [-0.2725599  -0.10313283]]

  [[-0.13886876 -0.2513148 ]
   [-0.25815955  0.2037502 ]]

  [[ 0.18243036  0.06070128]
   [ 0.0784964  -0.14904013]]]] 

--------
[[[ 0.21070072  0.12532869]
  [-0.2725599  -0.10313283]]

 [[-0.13886876 -0.2513148 ]
  [-0.25815955  0.2037502 ]]

 [[ 0.18243036  0.06070128]
  [ 0.0784964  -0.14904013]]] 

---------
[[[ 0.18838969 -0.20607772]
  [-0.04675874 -0.2579743 ]]

 [[-0.13886876 -0.2513148 ]
  [-0.25815955  0.2037502 ]]] 

---------


# Channel Pruning in PyTorch

PyTorch is great. Tensorflow sucks for this kind of thing (https://stackoverflow.com/questions/40295700/how-to-assign-values-to-a-subset-of-a-tensor-in-tensorflow/48544723#48544723) 

Let's use all that PyTorchy slicing stuff.

Start with fully convolutional network:

In [120]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv3 = nn.Conv2d(16, 16, 5)
        self.conv4 = nn.Conv2d(16, 10, 5)
        self.fc1   = nn.Linear(10*5*5, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv3(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv4(out))
        out = F.max_pool2d(out, 2)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
    
class TinyNet(nn.Module):
    def __init__(self, channels):
        super(TinyNet, self).__init__()
        self.conv1 = nn.Conv2d(3, channels[0], 5)
        self.conv2 = nn.Conv2d(channels[0], channels[1], 5)
        self.conv3 = nn.Conv2d(channels[1], channels[2], 5)
        self.conv4 = nn.Conv2d(channels[2], channels[3], 5)
        self.fc1   = nn.Linear(channels[3] * 5 * 5, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv3(out))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv4(out))
        out = F.max_pool2d(out, 2)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    

def calculate_threshold(weights, ratio):
    return np.percentile(np.array(torch.abs(weights)), ratio)

def sparsify(model, threshold=99.):
    for name, param in model.named_parameters():
        if 'weight' in name:
            param.data = (param * torch.gt(torch.abs(param), calculate_threshold(param.data, threshold)).float()).data
    return model

def argwhere_nonzero(layer):
    indices=[]

    for idx,w in enumerate(layer):
        if torch.sum(torch.abs(w)) != 0.:
            indices.append(idx)
    
    return indices


def prune_conv(indices, layer, follow=False):
    if not follow:
        layer.weight.data = layer.weight[indices].data
        layer.bias.data   = layer.bias[indices].data
    else: 
        layer.weight.data = layer.weight[:,indices].data

        
        
def compress_convs(model):
    ls = list(model.children())

    channels = []
    nonzeros = []
    for l1, l2 in zip(ls, ls[1:]):
        # so now we have pairs of layers
        if isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d):
            # if both conv - prune the first and then remove inputs from second
            nonzeros = argwhere_nonzero(l1.weight)

            prune_conv(nonzeros, l1)
            prune_conv(nonzeros, l2, follow=True)
            
            channels.append(len(nonzeros))
            nonzeros = argwhere_nonzero(l2.weight) # discarded until last iteration
            
        
    # append last layer size
    channels.append(len(nonzeros))
    
    return channels
            

model = LeNet()    
sparsify(model)
channels = compress_convs(model)


new_model = TinyNet(channels)

for original, compressed in zip(model.children(), new_model.children()):
    compressed.weight = original.weight
    compressed.bias   = original.bias
    
new_model

TinyNet(
  (conv1): Conv2d (3, 3, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (3, 7, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d (7, 13, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d (13, 9, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=225, out_features=10)
)